In [8]:
%pip install numpy
import numpy as np
from collections import Counter
import string
import re

Note: you may need to restart the kernel to use updated packages.


# 3.1 Hill Cipher

In [9]:
def text_to_numbers(text):
    return [ord(c) - ord('A') for c in text.upper() if c in string.ascii_uppercase]

def numbers_to_text(numbers):
    return ''.join([chr(int(n) % 26 + ord('A')) for n in numbers])

def mod_inverse_matrix(matrix, modulus=26):
    det = int(np.round(np.linalg.det(matrix))) % modulus
    if np.gcd(det, modulus) != 1: return None
    det_inv = pow(det, -1, modulus)
    matrix_adj = np.round(det * np.linalg.inv(matrix)).astype(int)
    return (det_inv * matrix_adj) % modulus

def encrypt(text, key_matrix):
    n = len(key_matrix)
    numbers = text_to_numbers(text)
    while len(numbers) % n != 0: numbers.append(23)
    cipher_numbers = []
    for i in range(0, len(numbers), n):
        block = np.array(numbers[i:i+n])
        cipher_numbers.extend(np.dot(key_matrix, block) % 26)
    return numbers_to_text(cipher_numbers)

def decrypt(text, key_matrix):
    inv = mod_inverse_matrix(key_matrix)
    return encrypt(text, inv) if inv is not None else None

def brute_force_2x2(ciphertext, known_plain=''):
    results = []
    for a in range(26):
        for b in range(26):
            for c in range(26):
                for d in range(26):
                    key = np.array([[a,b],[c,d]])
                    det = (a*d - b*c) % 26
                    if np.gcd(det, 26) != 1: continue
                    plain = decrypt(ciphertext, key)
                    if plain and (not known_plain or known_plain.upper() in plain):
                        results.append((key, plain))
    return results

key = np.array([[3, 3], [2, 5]])
plaintext = 'ACHU'
ciphertext = encrypt(plaintext, key)
print(f'Plain: {plaintext}, Cipher: {ciphertext}')
print(f'Decrypted: {decrypt(ciphertext, key)}')
print(f'Brute Force: {brute_force_2x2(ciphertext, "ACHU")}')


Plain: ACHU, Cipher: GKDK
Decrypted: ACHU
Brute Force: [(array([[3, 3],
       [2, 5]]), 'ACHU')]


# 3.2 Frequency Analysis (Book)

In [10]:
with open('book.txt', 'r', encoding='utf-8') as f:
    content = f.read()

sample = content[:50000]  # Sample for fast scoring
common = {'the','and','to','of','a','in','that','is','was','he','for','it','with','as','his','on','be','at','by','i','this','had','not','are','but','from','or','have','an','they','which','one','you','were','her','all','she','there','would','their','we','him','been','has','when','who','will','more','no','if','out','so','said','what','up','its','about','into','than','them','can','only','other','new','some','could','time','these','two','may','then','do','first','any','my','now','such','like','our','over','man','me','even','most','made','after','also','did','many','before','must','through','back','years','where','much','your','way','well','down','should','because','each','just','those','people','how','too','little','state','good','very','make','world','still','own','see','men','work','long','get','here','between','both','life','being','under','never','day','same','another','know','while','last','might','us','great','old','year','off','come','since','against','go','came','right','used','take','three'}

ignore = set(' \n\r,.!?;:—\'"()-*0123456789')
cipher_chars = [c for c in content if c not in ignore and ord(c) > 127]
cipher_freq = [c for c, _ in Counter(cipher_chars).most_common(26)]
eng = list('etaoinhsrdlucmwfgypbvkjxqz')

def dec(m, t): return ''.join(m.get(c, c) for c in t)
def score(m): return sum(1 for w in re.findall(r'[a-z]+', dec(m, sample).lower()) if w in common)

mapping = dict(zip(cipher_freq, eng))
best = score(mapping)
for _ in range(3):  # Multiple passes
    for i in range(26):
        for j in range(i+1, 26):
            eng[i], eng[j] = eng[j], eng[i]
            m = dict(zip(cipher_freq, eng))
            if score(m) > best:
                best, mapping = score(m), m
            else:
                eng[i], eng[j] = eng[j], eng[i]

print(dec(mapping, content)[:500])

stately, plump buck mulligan came from the stairhead, bearing a bowl of lather on which a mirror and a ra🤓or lay crossed. a yellow dressinggown, ungirdled, was sustained gently behind him on the mild morning air. he held the bowl aloft and intoned:

—introibo ad altare dei.

halted, he peered down the dark winding stairs and called out coarsely:

—come up, kinch! come up, you fearful jesuit!

solemnly he came forward and mounted the round gunrest. he faced about and blessed gravely thrice the to


In [ ]:
# We were able to get all the emoji to letter mapping from the above
# (More than enough to find the book and the author, we're doing this for the style points)
EMOJI_TO_LETTER = {
    '😙': 'a', '🤪': 'b', '😎': 'c', '🥳': 'd', '😍': 'e', '😃': 'f', '🤩': 'g',
    '😢': 'h', '👂': 'i', '😒': 'j', '😂': 'k', '🙀': 'l', '😡': 'm', '🥴': 'n',
    '🥶': 'o', '😵': 'p', '👌': 'q', '😉': 'r', '🥺': 's', '🤡': 't', '👧': 'u',
    '🦣': 'v', '🙄': 'w', '🫠': 'x', '🤐': 'y', '🤓': 'z'
}
def decode(content):
    return ''.join(EMOJI_TO_LETTER.get(c, c) for c in content)

print(decode(content)[-200:])

 flower and first i put my arms around him yes and drew him down to me so he could feel my breasts all perfume yes and his heart was going like mad and yes i said yes i will yes.

trieste-zurich-paris


# 3.3 Spiral Cipher

In [12]:
text = 'taskphaWL_PL4sOingpYefdngaP{__diddL40ap}y5rn_s1m37'[:49]
rows, cols = 7, 7
grid = [list(text[r*cols:(r+1)*cols]) for r in range(rows)]

top, bottom, left, right = 0, 6, 0, 6
result = []
while top <= bottom and left <= right:
    for i in range(left, right + 1): result.append(grid[top][i])
    top += 1
    for i in range(top, bottom + 1): result.append(grid[i][right])
    right -= 1
    if top <= bottom:
        for i in range(right, left - 1, -1): result.append(grid[bottom][i])
        bottom -= 1
    if left <= right:
        for i in range(bottom, top - 1, -1): result.append(grid[i][left])
        left += 1

print(f"Flag: {''.join(result)}")

Flag: taskphase{L53m1s_nr4_fOWL_PL4YPdy}pa0_dingpadidng


# 3.4 Average Cipher

In [14]:
# TBD